## CML Project Administration Examples

#### You can use the API to manage your CML projects

In [1]:
import cmlapi
import os
config = cmlapi.Configuration()

In [2]:
config.host = os.environ["PROJECT_HOST"]

In [3]:
client = cmlapi.ApiClient(config)
client.set_default_header("authorization", "Bearer "+os.environ["API_KEY"])
api_instance = cmlapi.CMLServiceApi(client)

#### You can see all projects contained in the CML Workspace

In [4]:
projects = api_instance.list_projects()

In [5]:
projects

{'next_page_token': '',
 'projects': [{'created_at': datetime.datetime(2021, 7, 10, 0, 12, 13, 355200, tzinfo=tzlocal()),
               'creation_status': 'success',
               'creator': {'email': 'pauldefusco@cloudera.com',
                           'id': '1',
                           'name': 'Paul de Fusco',
                           'username': 'pauldefusco'},
               'default_project_engine_type': 'ml_runtime',
               'description': '',
               'full_name': 'test',
               'id': 'llgb-lqu2-7ivf-is2r',
               'name': 'test',
               'owner': {'email': 'pauldefusco@cloudera.com',
                         'id': '1',
                         'name': 'Paul de Fusco',
                         'username': 'pauldefusco'},
               'project_visibility': 'public',
               'updated_at': datetime.datetime(2021, 7, 18, 20, 32, 3, 354805, tzinfo=tzlocal()),
               'user_updated_at': ''},
              {'created_at': datet

#### The projects object is not a Python dictionary but you can transform it into one so you can extract all the info you need

In [6]:
type(projects)

cmlapi.models.list_projects_response.ListProjectsResponse

In [7]:
proj_dict = projects.to_dict()

In [8]:
print("Number of Projects Created")
len(proj_dict["projects"])

Number of Projects Created


2

#### Each project contains information about project creation, author, etc.

In [9]:
proj_dict["projects"][0]

{'id': 'llgb-lqu2-7ivf-is2r',
 'name': 'test',
 'full_name': 'test',
 'owner': {'id': '1',
  'username': 'pauldefusco',
  'name': 'Paul de Fusco',
  'email': 'pauldefusco@cloudera.com'},
 'creator': {'id': '1',
  'username': 'pauldefusco',
  'name': 'Paul de Fusco',
  'email': 'pauldefusco@cloudera.com'},
 'description': '',
 'project_visibility': 'public',
 'default_project_engine_type': 'ml_runtime',
 'created_at': datetime.datetime(2021, 7, 10, 0, 12, 13, 355200, tzinfo=tzlocal()),
 'updated_at': datetime.datetime(2021, 7, 10, 0, 33, 43, 517504, tzinfo=tzlocal()),
 'user_updated_at': '',
 'creation_status': 'success'}

#### You can consolidate each project's info into a single dataframe and then query it to monitor activity

In [10]:
!pip3 install pandas
import pandas as pd
pd.DataFrame(proj_dict["projects"])

You should consider upgrading via the '/usr/local/bin/python3.6 -m pip install --upgrade pip' command.


,id,name,full_name,owner,creator,description,project_visibility,default_project_engine_type,created_at,updated_at,user_updated_at,creation_status
0,llgb-lqu2-7ivf-is2r,test,test,"{'id': '1', 'username': 'pauldefusco', 'name':...","{'id': '1', 'username': 'pauldefusco', 'name':...",,public,ml_runtime,2021-07-10 00:12:13.355200+00:00,2021-07-10 00:33:43.517504+00:00,,success
1,qqbn-sekc-qwme-4kyg,API V2 Demo,api-v2-demo,"{'id': '1', 'username': 'pauldefusco', 'name':...","{'id': '1', 'username': 'pauldefusco', 'name':...",,public,ml_runtime,2021-07-11 17:15:21.863715+00:00,2021-07-12 16:23:37.302979+00:00,,success


In [11]:
wspace_projs = pd.DataFrame(proj_dict["projects"])

In [12]:
wspace_projs[["full_name", "creator", "updated_at"]]

,full_name,creator,updated_at
0,test,"{'id': '1', 'username': 'pauldefusco', 'name':...",2021-07-10 00:33:43.517504+00:00
1,api-v2-demo,"{'id': '1', 'username': 'pauldefusco', 'name':...",2021-07-12 16:23:37.302979+00:00


#### For example, you can query the most recent project

In [50]:
from datetime import datetime

now = datetime.now()

In [58]:
wspace_projs[wspace_projs["updated_at"] == wspace_projs["updated_at"].min()]

,id,name,full_name,owner,creator,description,project_visibility,default_project_engine_type,created_at,updated_at,user_updated_at,creation_status
0,llgb-lqu2-7ivf-is2r,test,test,"{'id': '1', 'username': 'pauldefusco', 'name':...","{'id': '1', 'username': 'pauldefusco', 'name':...",,public,ml_runtime,2021-07-10 00:12:13.355200+00:00,2021-07-10 00:33:43.517504+00:00,,success


#### Or you can extract more info about the author

In [70]:
wspace_projs["owner"][0]

{'id': '1',
 'username': 'pauldefusco',
 'name': 'Paul de Fusco',
 'email': 'pauldefusco@cloudera.com'}

#### And put it back into the original dataframe

In [79]:
pd.concat([wspace_projs, pd.json_normalize(wspace_projs['owner'])], axis=1).drop(["owner", "creator"], axis=1)

,id,name,full_name,description,project_visibility,default_project_engine_type,created_at,updated_at,user_updated_at,creation_status,id,username,name,email
0,llgb-lqu2-7ivf-is2r,test,test,,public,ml_runtime,2021-07-10 00:12:13.355200+00:00,2021-07-10 00:33:43.517504+00:00,,success,1,pauldefusco,Paul de Fusco,pauldefusco@cloudera.com
1,qqbn-sekc-qwme-4kyg,API V2 Demo,api-v2-demo,,private,ml_runtime,2021-07-11 17:15:21.863715+00:00,2021-07-11 17:20:01.248690+00:00,,success,1,pauldefusco,Paul de Fusco,pauldefusco@cloudera.com
